In [1]:
from urllib.request import urlretrieve
import os
import numpy as np
import matplotlib.pyplot as pyplot
%matplotlib inline  
import sys


sys.path.insert(0, './datasets/')
from load_tables import load_ICM_csr, load_URM_csr
ICM_all = load_ICM_csr()
URM_all = load_URM_csr()

from Notebooks_utils.evaluation_function import evaluate_algorithm
from Notebooks_utils.data_splitter import train_test_holdout
from Base.Evaluation.Evaluator import EvaluatorHoldout
from myRecommenders.UserIcmKNNCFRecommender import UserIcmKNNCFRecommender



# User ICM

In [11]:
#SERVONO TUTTI E DUE
URM_train_validation, URM_test = train_test_holdout(URM_all, train_perc = 0.8)
URM_train, URM_validation = train_test_holdout(URM_train_validation, train_perc = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

from skopt.space import Real, Integer, Categorical
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 200
n_random_starts = int(n_cases*0.25)
metric_to_optimize = "MAP"

#qui devi scrivere che classe usi per fare tuning
#OCCHIO! la classe deve essere estensione di Base Recommender e implementare i soliti metodi fit,recommend ecc
recommender_class=UserIcmKNNCFRecommender

In [12]:
# questi licambi con i parametri che devi passare al fit, metti tra virgolette il nome del param e poi il range
hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["topK"] = Integer(1,2000) 
hyperparameters_range_dictionary["shrink"] = Real(0.1,100) 
hyperparameters_range_dictionary["normalize"] = Categorical([True, False]) 


recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train,ICM_all],     #qui metti quello che gli serve come "sorgente" dei dati
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)


recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],    #qui quello che usi x validation
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

 

In [13]:


import time
start_time = time.time()
parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation,
                                 evaluator_test=evaluator_test)


parameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,#scrive i risultati man mano su questo file
                       metric_to_optimize = metric_to_optimize
                      )

end_time = time.time()

print("Ha finito in questo tempo: "+str(end_time-start_time)+" secondi")

Iteration No: 1 started. Evaluating function at random point.
UserKNNCFRecommender: URM Detected 228 (2.87 %) cold users.
UserKNNCFRecommender: URM Detected 4029 (15.51 %) cold items.
UserKNNCFRecommender: URM Detected 229 (2.88 %) cold users.
UserKNNCFRecommender: URM Detected 91 (0.46 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 331, 'shrink': 6.969016739607128, 'normalize': True}
Similarity column 7947 ( 100 % ), 19403.58 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 3413.10 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 5167 ( 100.00% ) in 3.41 sec. Users per second: 1516
SearchBayesianSkopt: New best config found. Config 0: {'topK': 331, 'shrink': 6.969016739607128, 'normalize': True} - results: ROC_AUC: 0.1197472, PRECISION: 0.0236501, PRECISION_RECALL_MIN_DEN: 0.0938496, RECALL: 0.0920504, MAP: 0.0399025, MRR: 0.0842873, NDCG: 0.0624368, F1: 0.0376316, HIT_RATE: 0.2365009, ARHR: 0.0923536, NOVELTY: 0.0046640, AVERAGE_POPULARIT

Similarity column 7947 ( 100 % ), 19774.02 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 3800.40 column/sec, elapsed time 0.03 min
EvaluatorHoldout: Processed 5167 ( 100.00% ) in 2.80 sec. Users per second: 1846
SearchBayesianSkopt: Config 5 is suboptimal. Config: {'topK': 124, 'shrink': 90.44590644841789, 'normalize': False} - results: ROC_AUC: 0.0912153, PRECISION: 0.0173408, PRECISION_RECALL_MIN_DEN: 0.0616055, RECALL: 0.0601269, MAP: 0.0270287, MRR: 0.0632810, NDCG: 0.0428106, F1: 0.0269183, HIT_RATE: 0.1734082, ARHR: 0.0693819, NOVELTY: 0.0052126, AVERAGE_POPULARITY: 0.0613713, DIVERSITY_MEAN_INTER_LIST: 0.9713555, DIVERSITY_HERFINDAHL: 0.9971168, COVERAGE_ITEM: 0.4092782, COVERAGE_ITEM_CORRECT: 0.0226372, COVERAGE_USER: 0.6501825, COVERAGE_USER_CORRECT: 0.0934944, DIVERSITY_GINI: 0.1491620, SHANNON_ENTROPY: 11.4798063, 

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 5.6321
Function value obtained: -0.0270
Current minimum: -0.0399
Iterat

Similarity column 7947 ( 100 % ), 19685.57 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 3336.60 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 5167 ( 100.00% ) in 3.45 sec. Users per second: 1499
SearchBayesianSkopt: Config 11 is suboptimal. Config: {'topK': 544, 'shrink': 44.217047163727166, 'normalize': False} - results: ROC_AUC: 0.0932071, PRECISION: 0.0177085, PRECISION_RECALL_MIN_DEN: 0.0723409, RECALL: 0.0710516, MAP: 0.0309892, MRR: 0.0652837, NDCG: 0.0482384, F1: 0.0283510, HIT_RATE: 0.1770853, ARHR: 0.0707133, NOVELTY: 0.0047957, AVERAGE_POPULARITY: 0.1252665, DIVERSITY_MEAN_INTER_LIST: 0.9750349, DIVERSITY_HERFINDAHL: 0.9974846, COVERAGE_ITEM: 0.3293936, COVERAGE_ITEM_CORRECT: 0.0181713, COVERAGE_USER: 0.6501825, COVERAGE_USER_CORRECT: 0.0980244, DIVERSITY_GINI: 0.0907959, SHANNON_ENTROPY: 10.8074415, 

Iteration No: 12 ended. Evaluation done at random point.
Time taken: 6.6676
Function value obtained: -0.0310
Current minimum: -0.0399
Ite

Similarity column 7947 ( 100 % ), 21193.66 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 2992.59 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 5167 ( 100.00% ) in 4.34 sec. Users per second: 1191
SearchBayesianSkopt: Config 17 is suboptimal. Config: {'topK': 1303, 'shrink': 27.673640028817694, 'normalize': False} - results: ROC_AUC: 0.0809819, PRECISION: 0.0152893, PRECISION_RECALL_MIN_DEN: 0.0658830, RECALL: 0.0646059, MAP: 0.0268960, MRR: 0.0553971, NDCG: 0.0420540, F1: 0.0247269, HIT_RATE: 0.1528934, ARHR: 0.0596352, NOVELTY: 0.0045021, AVERAGE_POPULARITY: 0.1880121, DIVERSITY_MEAN_INTER_LIST: 0.9321268, DIVERSITY_HERFINDAHL: 0.9931946, COVERAGE_ITEM: 0.2241386, COVERAGE_ITEM_CORRECT: 0.0126660, COVERAGE_USER: 0.6501825, COVERAGE_USER_CORRECT: 0.0861960, DIVERSITY_GINI: 0.0433935, SHANNON_ENTROPY: 9.4165328, 

Iteration No: 18 ended. Evaluation done at random point.
Time taken: 8.0133
Function value obtained: -0.0269
Current minimum: -0.0399
Ite

Similarity column 7947 ( 100 % ), 21496.75 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 2819.70 column/sec, elapsed time 0.05 min
EvaluatorHoldout: Processed 5167 ( 100.00% ) in 4.37 sec. Users per second: 1181
SearchBayesianSkopt: Config 23 is suboptimal. Config: {'topK': 1380, 'shrink': 51.1582824738233, 'normalize': True} - results: ROC_AUC: 0.0939564, PRECISION: 0.0182117, PRECISION_RECALL_MIN_DEN: 0.0715821, RECALL: 0.0699257, MAP: 0.0306323, MRR: 0.0652346, NDCG: 0.0477735, F1: 0.0288973, HIT_RATE: 0.1821173, ARHR: 0.0712387, NOVELTY: 0.0043464, AVERAGE_POPULARITY: 0.2211540, DIVERSITY_MEAN_INTER_LIST: 0.9138571, DIVERSITY_HERFINDAHL: 0.9913680, COVERAGE_ITEM: 0.1718191, COVERAGE_ITEM_CORRECT: 0.0133205, COVERAGE_USER: 0.6501825, COVERAGE_USER_CORRECT: 0.0991569, DIVERSITY_GINI: 0.0267766, SHANNON_ENTROPY: 8.7853990, 

Iteration No: 24 ended. Evaluation done at random point.
Time taken: 8.1820
Function value obtained: -0.0306
Current minimum: -0.0399
Iterat

...

Similarity column 7947 ( 100 % ), 20378.89 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 3647.66 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 5167 ( 100.00% ) in 3.00 sec. Users per second: 1725
SearchBayesianSkopt: Config 189 is suboptimal. Config: {'topK': 176, 'shrink': 0.1, 'normalize': True} - results: ROC_AUC: 0.1238857, PRECISION: 0.0251016, PRECISION_RECALL_MIN_DEN: 0.0973413, RECALL: 0.0953624, MAP: 0.0408756, MRR: 0.0879202, NDCG: 0.0646058, F1: 0.0397422, HIT_RATE: 0.2510161, ARHR: 0.0965309, NOVELTY: 0.0047781, AVERAGE_POPULARITY: 0.1067397, DIVERSITY_MEAN_INTER_LIST: 0.9861419, DIVERSITY_HERFINDAHL: 0.9985951, COVERAGE_ITEM: 0.2810780, COVERAGE_ITEM_CORRECT: 0.0256015, COVERAGE_USER: 0.6501825, COVERAGE_USER_CORRECT: 0.1325028, DIVERSITY_GINI: 0.0771063, SHANNON_ENTROPY: 10.9444365, 

Iteration No: 190 ended. Search finished for the next optimal point.
Time taken: 8.9891
Function value obtained: -0.0409
Current minimum: -0.0418
Iterat

Similarity column 7947 ( 100 % ), 19813.96 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 2843.51 column/sec, elapsed time 0.05 min
EvaluatorHoldout: Processed 5167 ( 100.00% ) in 4.07 sec. Users per second: 1268
SearchBayesianSkopt: Config 195 is suboptimal. Config: {'topK': 1018, 'shrink': 0.1, 'normalize': True} - results: ROC_AUC: 0.1067036, PRECISION: 0.0212115, PRECISION_RECALL_MIN_DEN: 0.0859000, RECALL: 0.0842552, MAP: 0.0342927, MRR: 0.0737806, NDCG: 0.0551573, F1: 0.0338909, HIT_RATE: 0.2121153, ARHR: 0.0806742, NOVELTY: 0.0043572, AVERAGE_POPULARITY: 0.2080627, DIVERSITY_MEAN_INTER_LIST: 0.9305477, DIVERSITY_HERFINDAHL: 0.9930368, COVERAGE_ITEM: 0.1622714, COVERAGE_ITEM_CORRECT: 0.0165159, COVERAGE_USER: 0.6501825, COVERAGE_USER_CORRECT: 0.1150120, DIVERSITY_GINI: 0.0260392, SHANNON_ENTROPY: 8.9519748, 

Iteration No: 196 ended. Search finished for the next optimal point.
Time taken: 11.1976
Function value obtained: -0.0343
Current minimum: -0.0418
Itera

C:\Users\aless\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Similarity column 7947 ( 100 % ), 17876.71 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 3422.28 column/sec, elapsed time 0.04 min
EvaluatorHoldout: Processed 5167 ( 100.00% ) in 2.97 sec. Users per second: 1738
SearchBayesianSkopt: Config 197 is suboptimal. Config: {'topK': 140, 'shrink': 0.1, 'normalize': True} - results: ROC_AUC: 0.1248145, PRECISION: 0.0249081, PRECISION_RECALL_MIN_DEN: 0.0961024, RECALL: 0.0940228, MAP: 0.0417959, MRR: 0.0888235, NDCG: 0.0650548, F1: 0.0393830, HIT_RATE: 0.2490807, ARHR: 0.0975168, NOVELTY: 0.0048277, AVERAGE_POPULARITY: 0.0986897, DIVERSITY_MEAN_INTER_LIST: 0.9883143, DIVERSITY_HERFINDAHL: 0.9988123, COVERAGE_ITEM: 0.2989413, COVERAGE_ITEM_CORRECT: 0.0265255, COVERAGE_USER: 0.6501825, COVERAGE_USER_CORRECT: 0.1317478, DIVERSITY_GINI: 0.0872977, SHANNON_ENTROPY: 11.1619095, 

Iteration No: 198 ended. Search finished for the next optimal point.
Time taken: 9.5773
Function value obtained: -0.0418
Current minimum: -0.0418
Iterat

TypeError: __init__() missing 1 required positional argument: 'ICM'

In [14]:
#qui legge da file i miglior pramateri trovati, 
# OCCHIO! non serve che finisca, puoi interrompere sopra e leggere comunque i parametri xk sono salvati sul file (RUNNANDO QUI)
from Base.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

best_parameters = search_metadata["hyperparameters_best"]
best_parameters
# c'è un bug e non printa i ture/False

{'topK': 139, 'shrink': 0.1}

In [15]:
# qui fai una run con il solito validation così puoi avere un idea della vera xformance
#sopra essendo più splittato avrai SEMPRE  map piu piccoli, circa di 0.01 o giù di li
URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

uICMRecommender=  UserIcmKNNCFRecommender(URM_train,ICM_all)
uICMRecommender.fit(**best_parameters)

result_dict, _ = evaluator_test.evaluateRecommender(uICMRecommender)
MAP = result_dict[10]["MAP"]
print("MAP user icm CF recommeder: "+str(MAP)+"\n")

UserKNNCFRecommender: URM Detected 67 (0.84 %) cold users.
UserKNNCFRecommender: URM Detected 2345 (9.03 %) cold items.
UserKNNCFRecommender: URM Detected 68 (0.86 %) cold users.
UserKNNCFRecommender: URM Detected 38 (0.19 %) cold items.
Similarity column 7947 ( 100 % ), 18971.37 column/sec, elapsed time 0.01 min
Similarity column 7947 ( 100 % ), 2894.06 column/sec, elapsed time 0.05 min
EvaluatorHoldout: Processed 5586 ( 100.00% ) in 3.27 sec. Users per second: 1710
MAP user icm CF recommeder: 0.05093319984038486

